In [ ]:
!pip install category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import re
import numpy as np
import pandas as pd
import warnings
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from scipy.stats import shapiro, kstest, pearsonr
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.neighbors import LocalOutlierFactor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelBinarizer
# import category_encoders as ce


warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/used_cars_data_korea.csv')

0         201 hp @ 6,000 RPM
1         147 hp @ 6,200 RPM
2         164 hp @ 6,200 RPM
3         164 hp @ 6,200 RPM
4         147 hp @ 6,200 RPM
                 ...        
248422                   NaN
248423    290 hp @ 6,400 RPM
248424    290 hp @ 6,400 RPM
248425    290 hp @ 6,400 RPM
248426    130 hp @ 6,300 RPM
Name: power, Length: 248427, dtype: object

In [ ]:
# 분석 시 사용 안하는 칼럼 제거
columns_to_drop = [
    'Unnamed: 0', 'city', 'dealer_zip', 'is_cpo', 'engine_cylinders',
    'is_oemcpo', 'latitude', 'listing_color', 'savings_amount', 'longitude',
    'listed_date', 'bed', 'model_name', 'owner_count', 'sp_id', 'sp_name',
    'trimId', 'trim_name', 'vehicle_damage_category', 'wheel_system_display',
    'bed_height', 'bed_length', 'combine_fuel_economy', 'cabin',
    'is_certified', 'vehicle_damage_category'
]

df = df.drop(columns_to_drop, axis=1)

In [ ]:
# 결측치 확인
df.isna().sum()

back_legroom             16422
body_type                 5824
city_fuel_economy        47155
daysonmarket                 0
engine_displacement      24930
engine_type               9878
exterior_color               0
fleet                   135494
frame_damaged           135494
franchise_dealer             0
franchise_make           43186
front_legroom            16422
fuel_tank_volume         16422
fuel_type                 8089
has_accidents           135494
height                   16422
highway_fuel_economy     47155
horsepower               24930
interior_color               7
isCab                   135494
is_new                       0
length                   16422
major_options            13655
make_name                    0
maximum_seating          16422
mileage                  12315
power                    35356
price                        0
salvage                 135494
seller_rating             4983
theft_title             135494
torque                   36165
transmis

In [ ]:
# 단위 제거
def del_str(col_name, str):
    df[col_name] = df[col_name].fillna('0' + str)
    df[col_name] = df[col_name].replace('--', '0'+str)

    df[col_name] = df[col_name].apply(lambda x: x.replace(str, '').strip())
    df[col_name] = df[col_name].astype(float)

del_str(col_name='back_legroom', str=' in')
del_str(col_name='front_legroom', str=' in')
del_str(col_name='height', str=' in')
del_str(col_name='length', str=' in')
del_str(col_name='wheelbase', str=' in')
del_str(col_name='width', str=' in')
del_str(col_name='fuel_tank_volume', str=' gal')
del_str(col_name='maximum_seating', str=' seats')

# 두개의 변수가 하나의 column에 존재하는 것 분리
pattern = r'(\d+)\s+lb-ft\s+@\s+(\d+),?(\d*)\s*RPM'

df['torque_lb_ft'] = df['torque'].str.extract(pattern)[0].astype(float)
df['torque_rpm'] = (df['torque'].str.extract(pattern)[1]+df['torque'].str.extract(pattern)[2]).astype(float)

df = df.drop('torque', axis=1)

pattern = r'(\d+)\s+hp\s+@\s+(\d+),?(\d*)\s*RPM'

df['power_hp'] = df['power'].str.extract(pattern)[0].astype(float)
df['power_rpm'] = (df['power'].str.extract(pattern)[1]+df['power'].str.extract(pattern)[2]).astype(float)

df = df.drop('power', axis=1)

In [ ]:
df.theft_title = df.theft_title.replace(to_replace={True:1,False:0})
df.frame_damaged = df.frame_damaged.replace(to_replace={True:1,False:0})
df.fleet = df.fleet.replace(to_replace={True:1,False:0})
df.franchise_dealer = df.franchise_dealer.replace(to_replace={True:1,False:0})
df.has_accidents = df.has_accidents.replace(to_replace={True:1,False:0})
df.isCab = df.isCab.replace(to_replace={True:1,False:0})
df.is_new = df.is_new.replace(to_replace={True:1,False:0})
df.salvage = df.salvage.replace(to_replace={True:1,False:0})

In [ ]:
df.isnull().sum()

back_legroom                 0
body_type                 5824
city_fuel_economy        47155
daysonmarket                 0
engine_displacement      24930
engine_type               9878
exterior_color               0
fleet                   135494
frame_damaged           135494
franchise_dealer             0
franchise_make           43186
front_legroom                0
fuel_tank_volume             0
fuel_type                 8089
has_accidents           135494
height                       0
highway_fuel_economy     47155
horsepower               24930
interior_color               7
isCab                   135494
is_new                       0
length                       0
major_options            13655
make_name                    0
maximum_seating              0
mileage                  12315
price                        0
salvage                 135494
seller_rating             4983
theft_title             135494
transmission              5818
transmission_display      5818
wheel_sy

In [ ]:
df = df.dropna(subset=['has_accidents'])
df= df.reset_index(drop=True)

In [ ]:
# 도메인 지식 이용하여 0값으로 결측치 채워져있는 것들 Null값으로 변환
df['back_legroom'] = df['back_legroom'].replace(0, np.nan)
df['daysonmarket'] = df['daysonmarket'].replace(0, np.nan)
df['front_legroom'] = df['front_legroom'].replace(0, np.nan)
df['fuel_tank_volume'] = df['fuel_tank_volume'].replace(0, np.nan)
df['height'] = df['height'].replace(0, np.nan)
df['length'] = df['length'].replace(0, np.nan)
df['wheelbase'] = df['wheelbase'].replace(0, np.nan)
df['width'] = df['width'].replace(0, np.nan)
df['wheelbase'] = df['wheelbase'].replace(0, np.nan)
df['maximum_seating'] = df['maximum_seating'].replace(0, np.nan)

df.select_dtypes(include=['number']).isnull().sum()

back_legroom             2916
city_fuel_economy       11333
daysonmarket             1604
engine_displacement      2779
fleet                       0
frame_damaged               0
franchise_dealer            0
front_legroom            2916
fuel_tank_volume         2917
has_accidents               0
height                   2916
highway_fuel_economy    11333
horsepower               2779
isCab                       0
is_new                      0
length                   2916
maximum_seating          2916
mileage                   699
price                       0
salvage                     0
seller_rating            2848
theft_title                 0
wheelbase                2916
width                    2916
year                        0
torque_lb_ft             8798
torque_rpm               8798
power_hp                 8070
power_rpm                8070
dtype: int64

In [ ]:
# 주행거리 결측치 699개 row 삭제
df = df.dropna(subset=['mileage'])
df= df.reset_index(drop=True)

In [ ]:
# 옵션 개수를 대체
df['major_options'] = df['major_options'].apply(lambda x: len(eval(x)) if pd.notnull(x) else np.nan)

# 년도 방식 수정
df['year'] =  2021 - df['year']

In [ ]:
# 범주가 너무 많은 것들 제외
df = df.drop(['exterior_color'],axis = 1)
df = df.drop(['franchise_make'],axis = 1)
df = df.drop(['interior_color'],axis = 1)
df = df.drop(['transmission_display'],axis = 1)

In [ ]:
# 범주형 변수 결측치 대체
categorical_columns = df.select_dtypes(exclude=['int64', 'float64']).columns

# 범주형 변수 자체를 만듬 NA
df1 = df
for col in categorical_columns:
    df1[col] = df1[col].fillna("Unknown")

# 범주형 변수 최빈값 대체
df2 = df
for col in categorical_columns:
    mode_value = df2[col].mode()[0]  # 최빈값 계산
    df2[col] = df2[col].fillna(mode_value)

# 그냥 범주형 변수 삭제
df3 = df
df3 = df3.drop(['engine_type'],axis = 1)
df3 = df3.drop(['make_name'],axis = 1)
df3 = df3.drop(['fuel_type'],axis = 1)
df3 = df3.drop(['transmission'],axis = 1)
df3 = df3.drop(['wheel_system'],axis = 1)
df3 = df3.drop(['body_type'],axis = 1)

Index(['body_type', 'engine_type', 'fuel_type', 'make_name', 'transmission',
       'wheel_system'],
      dtype='object')


In [ ]:
# 중앙값 대체
df_median1 = df1
df_median2 = df2
df_median3 = df3

# knn 대치
df_knn1 = df1
df_knn2 = df2
df_knn3= df3

# 다중 대치
df_multiple_imputation1 = df1
df_multiple_imputation2 = df2
df_multiple_imputation3 = df3

In [ ]:
# 중앙값 대체
# 대체할 연속형 변수 선택
columns_with_missing_values = ['back_legroom', 'city_fuel_economy', 'daysonmarket', 'engine_displacement',
                               'front_legroom', 'fuel_tank_volume', 'height', 'highway_fuel_economy',
                               'horsepower', 'length', 'maximum_seating', 'seller_rating', 'wheelbase',
                               'width', 'torque_lb_ft', 'torque_rpm', 'power_hp', 'power_rpm']

# 중앙값으로 결측치 대체
df_median1[columns_with_missing_values] = df_median1[columns_with_missing_values].fillna(df_median1[columns_with_missing_values].median())
df_median2[columns_with_missing_values] = df_median2[columns_with_missing_values].fillna(df_median1[columns_with_missing_values].median())
df_median3[columns_with_missing_values] = df_median3[columns_with_missing_values].fillna(df_median1[columns_with_missing_values].median())

In [ ]:
# knn 대치
# 대체할 연속형 변수 선택
columns_with_missing_values = ['back_legroom', 'city_fuel_economy', 'daysonmarket', 'engine_displacement',
                               'front_legroom', 'fuel_tank_volume', 'height', 'highway_fuel_economy',
                               'horsepower', 'length', 'maximum_seating', 'seller_rating', 'wheelbase',
                               'width', 'torque_lb_ft', 'torque_rpm', 'power_hp', 'power_rpm']

imputer = KNNImputer(n_neighbors=5)
df_knn1[columns_with_missing_values] = imputer.fit_transform(df_knn1[columns_with_missing_values])
df_knn2[columns_with_missing_values] = imputer.fit_transform(df_knn2[columns_with_missing_values])
df_knn3[columns_with_missing_values] = imputer.fit_transform(df_knn3[columns_with_missing_values])

In [ ]:
# 다중 대치
# 대체할 연속형 변수 선택
columns_with_missing_values = ['back_legroom', 'city_fuel_economy', 'daysonmarket', 'engine_displacement',
                               'front_legroom', 'fuel_tank_volume', 'height', 'highway_fuel_economy',
                               'horsepower', 'length', 'maximum_seating', 'seller_rating', 'wheelbase',
                               'width', 'torque_lb_ft', 'torque_rpm', 'power_hp', 'power_rpm']

# mice 기법을 사용하여 결측치 대체
imputer = IterativeImputer()
df_multiple_imputation1[columns_with_missing_values] = imputer.fit_transform(df_multiple_imputation1[columns_with_missing_values])
df_multiple_imputation2[columns_with_missing_values] = imputer.fit_transform(df_multiple_imputation2[columns_with_missing_values])
df_multiple_imputation3[columns_with_missing_values] = imputer.fit_transform(df_multiple_imputation3[columns_with_missing_values])

In [ ]:
# 이상치 제거
def remove_outliers_iqr(df, columns, multiplier=1.5):
    for col in columns:
        if df[col].dtype in ['int64', 'float64']:  # 숫자형 데이터인 경우에만 적용
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - multiplier * IQR
            upper_bound = Q3 + multiplier * IQR
            df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    return df

columns_with_outliers = df.select_dtypes(include=['int64', 'float64']).columns

df_median1 = remove_outliers_iqr(df_median1, columns_with_outliers)
df_median2 = remove_outliers_iqr(df_median2, columns_with_outliers)
df_median3 = remove_outliers_iqr(df_median3, columns_with_outliers)
df_knn1 = remove_outliers_iqr(df_knn1, columns_with_outliers)
df_knn2 = remove_outliers_iqr(df_knn2, columns_with_outliers)
df_knn3 = remove_outliers_iqr(df_knn3, columns_with_outliers)
df_multiple_imputation1 = remove_outliers_iqr(df_multiple_imputation1, columns_with_outliers)
df_multiple_imputation2 = remove_outliers_iqr(df_multiple_imputation2, columns_with_outliers)
df_multiple_imputation3 = remove_outliers_iqr(df_multiple_imputation3, columns_with_outliers)

In [ ]:
def MinMax(df):
    numeric_columns = df.select_dtypes(include=['int64', 'float64']).columns
    numeric_columns = numeric_columns[~numeric_columns.isin(['price'])]
    categorical_columns = df.select_dtypes(include=['object', 'category']).columns
    categorical_columns = categorical_columns.append(pd.Index(['price']))


    scaler = MinMaxScaler()
    df_scaled = pd.DataFrame(df)  # 스케일링된 데이터를 저장할 새로운 DataFrame 생성

    df_scaled[numeric_columns] = scaler.fit_transform(df[numeric_columns])  # 숫자형 열에만 스케일링 적용

    # 범주형 열은 그대로 유지
    df_scaled[categorical_columns] = df[categorical_columns]

    return df_scaled

df_median_scaled1 = MinMax(df_median1)
df_median_scaled2 = MinMax(df_median2)
df_median_scaled3 = MinMax(df_median3)
df_knn_scaled1 = MinMax(df_knn1)
df_knn_scaled2 = MinMax(df_knn2)
df_knn_scaled3 = MinMax(df_knn3)
df_multiple_imputation_scaled1 = MinMax(df_multiple_imputation1)
df_multiple_imputation_scaled2 = MinMax(df_multiple_imputation2)
df_multiple_imputation_scaled3 = MinMax(df_multiple_imputation3)

In [ ]:
# 범주형 변수 바이너리 인코딩
def BinaryEn(df):
  # 범주형 변수 선택
  categorical_columns = df.select_dtypes(include=['object', 'category']).columns
  encoder = ce.BinaryEncoder(cols=categorical_columns)
  # 바이너리 인코딩 수행
  df_encoded = encoder.fit_transform(df)

  return df_encoded

df_median_scaled1 = BinaryEn(df_median_scaled1)
df_median_scaled2 = BinaryEn(df_median_scaled2)
df_median_scaled3 = BinaryEn(df_median_scaled3)
df_knn_scaled1 = BinaryEn(df_knn_scaled1)
df_knn_scaled2 = BinaryEn(df_knn_scaled2)
df_knn_scaled3 = BinaryEn(df_knn_scaled3)
df_multiple_imputation_scaled1 = BinaryEn(df_multiple_imputation_scaled1)
df_multiple_imputation_scaled2 = BinaryEn(df_multiple_imputation_scaled2)
df_multiple_imputation_scaled3 = BinaryEn(df_multiple_imputation_scaled3)

In [ ]:
#df_median_scaled1 = df_median_scaled1.drop(['Unnamed: 0'], axis = 1)
df_median_scaled1 =df_median_scaled1.reset_index(drop = True)
df_median_scaled2 =df_median_scaled2.reset_index(drop = True)
df_median_scaled3 =df_median_scaled3.reset_index(drop = True)
df_knn_scaled1 =df_knn_scaled1.reset_index(drop = True)
df_knn_scaled2 =df_knn_scaled2.reset_index(drop = True)
df_knn_scaled3 =df_knn_scaled3.reset_index(drop = True)
df_multiple_imputation_scaled1 =df_multiple_imputation_scaled1.reset_index(drop = True)
df_multiple_imputation_scaled2 =df_multiple_imputation_scaled2.reset_index(drop = True)
df_multiple_imputation_scaled3 =df_multiple_imputation_scaled3.reset_index(drop = True)

In [ ]:
df_multiple_imputation_scaled2["price"]

0        15939.0
1         7859.0
2        14359.0
3        15503.0
4         8223.0
          ...   
51007    17881.0
51008    17871.0
51009    12998.0
51010    18034.0
51011    10998.0
Name: price, Length: 51012, dtype: float64

In [ ]:
df_multiple_imputation_scaled2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51012 entries, 0 to 51011
Data columns (total 46 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   back_legroom          51012 non-null  float64
 1   body_type_0           51012 non-null  int64  
 2   body_type_1           51012 non-null  int64  
 3   body_type_2           51012 non-null  int64  
 4   city_fuel_economy     51012 non-null  float64
 5   daysonmarket          51012 non-null  float64
 6   engine_displacement   51012 non-null  float64
 7   engine_type_0         51012 non-null  int64  
 8   engine_type_1         51012 non-null  int64  
 9   engine_type_2         51012 non-null  int64  
 10  fleet                 51012 non-null  float64
 11  frame_damaged         51012 non-null  float64
 12  franchise_dealer      51012 non-null  float64
 13  front_legroom         51012 non-null  float64
 14  fuel_tank_volume      51012 non-null  float64
 15  fuel_type_0        

In [ ]:
preprocessing_data = [df_median_scaled1, df_median_scaled2, df_median_scaled3,
                      df_knn_scaled1, df_knn_scaled2, df_knn_scaled3,
                     df_multiple_imputation_scaled1,
                      df_multiple_imputation_scaled2
                      df_multiple_imputation_scaled3]

output_directory = '/content/drive/MyDrive/'

# 반복문을 통해 데이터프레임을 CSV 파일로 저장
for i, df in enumerate(preprocessing_data):
    filename = f'preprocess_final{i+1}.csv'
    output_path = output_directory + filename
    df.to_csv(output_path)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/preprocess_final1.csv')
data = df.copy()
columns = ['back_legroom', 'width', 'wheelbase', 'length', 'height', 'front_legroom']
selected_data = data[columns]

# PCA 모델 생성 및 변환
pca = PCA(n_components=1)  # 주성분의 개수를 1로 설정하여 새로운 변수 생성
new_variable = pca.fit_transform(selected_data)
# 새로운 변수를 데이터프레임에 추가
data['size'] = new_variable
scaler = MinMaxScaler()
data['size'] = scaler.fit_transform(data[['size']])

# 선택된 행 삭제
data = data.drop("Unnamed: 0", axis = 1)
data = data.drop(columns, axis=1)

# 결과 확인
print(data.head())  # 새로운 변수가 추가된 데이터프레임 출력

   body_type_0  body_type_1  body_type_2  city_fuel_economy  daysonmarket  \
0            0            0            1           0.200000      0.130769   
1            0            1            0           0.400000      0.146154   
2            0            1            0           0.466667      0.123077   
3            0            0            1           0.133333      0.407692   
4            0            1            1           0.600000      0.084615   

   engine_displacement  engine_type_0  engine_type_1  engine_type_2  fleet  \
0             0.444444              0              0              1    0.0   
1             0.222222              0              0              1    0.0   
2             0.888889              0              0              1    0.0   
3             0.888889              0              0              1    1.0   
4             0.222222              0              0              1    0.0   

   ...  transmission_1  transmission_2  wheel_system_0  wheel_system

In [ ]:
data.to_csv("/content/drive/MyDrive/preprocess_real_final.csv")